In [360]:
import pandas as pd
from zipfile import ZipFile
import numpy as np
import os
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import neighbors, datasets
from sklearn.ensemble import RandomForestClassifier
#from sklearn.lda import LDA
from statsmodels.discrete.discrete_model import MNLogit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import sklearn.linear_model as linear_model

# Content:
* 1. Data Cleaning and Processing
* 2. Feature selection
* 3. Build the logit model to do the interpretation

# 1. Data Cleaning and Processing

## 1.1 tax data

Tax Data: CDC STATE System Tobacco Legislation - Tax： https://chronicdata.cdc.gov/Legislation/CDC-STATE-System-Tobacco-Legislation-Tax/2dwv-vfam

In [2]:
!curl https://chronicdata.cdc.gov/api/views/2dwv-vfam/rows.csv?accessType=DOWNLOAD > Tax.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.5M    0 33.5M    0     0  2748k      0 --:--:--  0:00:12 --:--:-- 3051k


In [3]:
#Pick the Cigeratte tax rate in 2017
tax = pd.read_csv('Tax.csv')
tax17 = tax[tax['Year']==2017]
tax17.head()

/Users/A123/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Year,Quarter,LocationAbbr,LocationDesc,TopicDesc,MeasureDesc,DataSource,ProvisionGroupDesc,ProvisionDesc,ProvisionValue,...,Comments,Enacted_Date,Effective_Date,GeoLocation,DisplayOrder,TopicTypeId,TopicId,MeasureId,ProvisionGroupID,ProvisionID
120339,2017,1,MN,Minnesota,Legislation - Tax Combustible Tobacco,Little Cigar,OSH,Restrictions,Little Cigar Tax ($ per pack of 20),3.0400,...,NaN,10/31/2016,1/1/2017,"(46.35564873600049, -94.79420050299967)",2,LEG,1010LEG,672LCG,10GRP,417
121546,2017,2,CA,California,Legislation - Tax Combustible Tobacco,Little Cigar,OSH,Restrictions,Little Cigar Tax ($ per pack of 20),2.8700,...,Currently products labeled as little or small ...,10/31/2016,4/1/2017,"(37.63864012300047, -120.99999953799971)",2,LEG,1010LEG,672LCG,10GRP,417
121547,2017,2,CA,California,Legislation - Tax Combustible Tobacco,Cigarette,OSH,Restrictions,Cigarette Tax ($ per pack),2.870,...,NaN,10/31/2016,4/1/2017,"(37.63864012300047, -120.99999953799971)",5,LEG,1010LEG,670CGR,10GRP,411
121548,2017,1,MI,Michigan,Legislation - Tax Stamp,Tax Stamp,OSH,Requirements,Barcode/Scannable Code Required,Yes,...,NaN,6/20/2012,6/20/2012,"(44.6613195430005, -84.71439026999968)",2,LEG,1030LEG,690CET,12GRP,350
121549,2017,1,RI,Rhode Island,Legislation - Tax Non-Combustible Tobacco,Dry Snuff Tobacco,OSH,Restrictions,Dry Snuff Tobacco Tax,Yes,...,NaN,1/1/1995,1/1/1995,"(41.70828019300046, -71.52247031399963)",1,LEG,1020LEG,682EDS,10GRP,392


In [46]:
tax_value = tax17[tax17['ProvisionDesc'] == 'Cigarette Tax ($ per pack)'].sort_values('LocationDesc')
tax_value = tax_value.groupby('LocationDesc').max()
tax_value = tax_value[['ProvisionAltValue']]
tax_value.rename(columns={'ProvisionAltValue':'Cigarette_tax'}, inplace=True)
tax_value.reset_index(inplace=True)
tax_value.head()

,LocationDesc,Cigarette_tax
0,Alabama,0.675
1,Alaska,2.000
2,American Samoa,6.000
3,Arizona,2.000
4,Arkansas,1.150


## 1.2 BRFSS data

2017 BRFSS Survey Data and Documentation https://www.cdc.gov/brfss/annual_data/annual_2017.html

In [5]:
#!curl https://www.cdc.gov/brfss/annual_data/2017/files/LLCP2017XPT.zip > healthdata17.zip

In [6]:
file = 'llcp2017_formatted.csv'
df = pd.read_csv(file)

/Users/A123/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,170,174) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,STATE FIPS CODE,FILE MONTH,INTERVIEW DATE,INTERVIEW MONTH,INTERVIEW DAY,INTERVIEW YEAR,FINAL DISPOSITION,ANNUAL SEQUENCE NUMBER,PRIMARY SAMPLING UNIT,CORRECT TELEPHONE NUMBER?,...,_IMPCAGE,_IMPCRAC,_IMPCSEX,_IMPEDUC,_IMPHOME,_IMPMRTL,_IMPNPH,_IMPSEX,_M_RACE,_URBNRRL
0,Alabama,January,1302017,01,30,2017,1100,2017000001,2017000001,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
1,Alabama,January,1122017,01,12,2017,1100,2017000002,2017000002,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
2,Alabama,January,1102017,01,10,2017,1100,2017000003,2017000003,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
3,Alabama,January,2082017,02,8,2017,1200,2017000004,2017000004,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
4,Alabama,January,1302017,01,30,2017,1100,2017000005,2017000005,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing


### The computed and calculated columns are cleaned data and have fewer missing values

In [58]:
filterList = ['COMPUTED', 'CALCULATED']
def containsKeyword(sent):
    for f in filterList:
        if f in sent:
            return True
    return False
    
cols41 = [c for c in df.columns if containsKeyword(c)]
print("Number of calculated/computed columns: ",len(cols41))
cols41

Number of calculated/computed columns:  41


['COMPUTED PHYSICAL HEALTH STATUS',
 'COMPUTED MENTAL HEALTH STATUS',
 'HIGH BLOOD PRESSURE CALCULATED VARIABLE',
 'CHOLESTEROL CHECKED CALCULATED VARIABLE',
 'HIGH CHOLESTEROL CALCULATED VARIABLE',
 'LIFETIME ASTHMA CALCULATED VARIABLE',
 'CURRENT ASTHMA CALCULATED VARIABLE',
 'COMPUTED ASTHMA STATUS',
 'COMPUTED PREFERRED RACE',
 'CALCULATED NON-HISPANIC RACE INCLUDING M',
 'COMPUTED RACE-ETHNICITY GROUPING',
 'COMPUTED NON-HISPANIC WHITES/ALL OTHERS',
 'COMPUTED FIVE LEVEL RACE/ETHNICITY CATEG',
 'COMPUTED RACE GROUPS USED FOR INTERNET P',
 'COMPUTED HEIGHT IN INCHES',
 'COMPUTED HEIGHT IN METERS',
 'COMPUTED WEIGHT IN KILOGRAMS',
 'COMPUTED BODY MASS INDEX',
 'COMPUTED BODY MASS INDEX CATEGORIES',
 'OVERWEIGHT OR OBESE CALCULATED VARIABLE',
 'COMPUTED NUMBER OF CHILDREN IN HOUSEHOLD',
 'COMPUTED LEVEL OF EDUCATION COMPLETED CA',
 'COMPUTED INCOME CATEGORIES',
 'COMPUTED SMOKING STATUS',
 'CURRENT SMOKING CALCULATED VARIABLE',
 'COMPUTED E-CIGARETTE USER STATUS',
 'CURRENT E-CIGARET

In [ ]:
cols = cols41.copy()
cols.append('STATE FIPS CODE')

In [62]:
df_use = df[cols]
df_use.shape

(450642, 42)

## 1.3 Merge the tax data with smoke data

In [273]:
df_cut = pd.merge(df_use, tax_value, left_on='STATE FIPS CODE', right_on='LocationDesc')

In [274]:
df_cut.shape

(450642, 44)

In [275]:
df_cut.drop(['LocationDesc'], axis=1, inplace=True)

In [276]:
df_cut.shape

(450642, 43)

In [277]:
df_cut.head()

,COMPUTED PHYSICAL HEALTH STATUS,COMPUTED MENTAL HEALTH STATUS,HIGH BLOOD PRESSURE CALCULATED VARIABLE,CHOLESTEROL CHECKED CALCULATED VARIABLE,HIGH CHOLESTEROL CALCULATED VARIABLE,LIFETIME ASTHMA CALCULATED VARIABLE,CURRENT ASTHMA CALCULATED VARIABLE,COMPUTED ASTHMA STATUS,COMPUTED PREFERRED RACE,CALCULATED NON-HISPANIC RACE INCLUDING M,...,COMPUTED DARK GREEN VEGETABLE INTAKE IN,COMPUTED POTATO SERVINGS PER DAY,COMPUTED OTHER VEGETABLE INTAKE IN TIMES,150 MINUTE PHYSICAL ACTIVITY CALCULATED,300 MINUTE PHYSICAL ACTIVITY CALCULATED,FLU SHOT CALCULATED VARIABLE,PNEUMONIA VACCINATION CALCULATED VARIABL,EVER BEEN TESTED FOR HIV CALCULATED VARI,STATE FIPS CODE,Cigarette_tax
0,Zero days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,Yes,No,No,Never,White,White only,...,43.0,14.0,100.0,150+ minutes (or vigorous equivalent minutes) ...,301+ minutes (or vigorous equivalent minutes)...,Yes,Yes,Yes,Alabama,0.675
1,Zero days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,No,No,No,Never,White,White only,...,100.0,0.0,100.0,150+ minutes (or vigorous equivalent minutes) ...,301+ minutes (or vigorous equivalent minutes)...,Yes,Yes,Yes,Alabama,0.675
2,Zero days when physical health not good,Zero days when mental health not good,No,Had cholesterol checked in past 5 years,Yes,No,No,Never,White,White only,...,14.0,0.0,14.0,0 minutes (or vigorous equivalent minutes) of ...,0 minutes (or vigorous equivalent minutes) of...,Yes,Yes,No,Alabama,0.675
3,Zero days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,Yes,No,No,Never,White,White only,...,NaN,NaN,NaN,Don�t know/Not Sure/Refused/Missing,Don�t know/Not Sure/Refused/Missing,Don�t know/Not Sure Or Refused/Missing,Don�t know/Not Sure Or Refused/Missing,Not asked or missing,Alabama,0.675
4,14+ days when physical health not good,Zero days when mental health not good,No,Had cholesterol checked in past 5 years,No,Yes,Yes,Current,White,White only,...,14.0,14.0,10.0,0 minutes (or vigorous equivalent minutes) of ...,0 minutes (or vigorous equivalent minutes) of...,No,No,No,Alabama,0.675


### Create the new feature (smoke_status) to indicate the smoker/e-smoker/both/none

In [278]:
#ecigdf = Data41.copy()
df_cut.rename(columns={'CURRENT E-CIGARETTE USER CALCULATED VARI': 'E_SMOKE', \
                     'CURRENT SMOKING CALCULATED VARIABLE':'SMOKE'},inplace = True)


mapper = {'Current E-cigarette user': 'Yes', 'Not currently using E-cigarettes': 'No' }
df_cut['E_SMOKE'] = df_cut['E_SMOKE'].map(mapper)
print("data size: ",len(df_cut))
df_cut = df_cut[df_cut['E_SMOKE'].apply(lambda x: x in ['Yes','No']) ]
df_cut = df_cut[df_cut['E_SMOKE'].apply(lambda x: x in ['Yes','No']) ]
print("clean data size: ",len(df_cut))
print("Frequency distributions:")
print(df_cut['E_SMOKE'].value_counts())
print(df_cut['SMOKE'].value_counts())

data size:  450642
clean data size:  429964
Frequency distributions:
No     416275
Yes     13689
Name: E_SMOKE, dtype: int64
No                            364794
Yes                            62732
Don�t know/Refused/Missing      2438
Name: SMOKE, dtype: int64


In [279]:
df_cut.loc[(df_cut['SMOKE']=='Yes')&(df_cut['E_SMOKE']=='Yes'),'smoke_type']='B'
df_cut.loc[(df_cut['SMOKE']=='No')&(df_cut['E_SMOKE']=='Yes'),'smoke_type']='E'
df_cut.loc[(df_cut['SMOKE']=='Yes')&(df_cut['E_SMOKE']=='No'),'smoke_type']='S'
df_cut.loc[(df_cut['SMOKE']=='No')&(df_cut['E_SMOKE']=='No'),'smoke_type']='N'
df_cut['smoke_type'].value_counts()

N    358383
S     55523
B      7209
E      6411
Name: smoke_type, dtype: int64

In [280]:
# Find the columns that contain original smoking key word
filterList = ['SMOK','CIG']
def containsKeyword(sent):
    for f in filterList:
        if f in sent:
            return True
    return False
    
smokecols = [c for c in df_cut.columns if containsKeyword(c)]
smokecols

['COMPUTED SMOKING STATUS',
 'SMOKE',
 'COMPUTED E-CIGARETTE USER STATUS',
 'E_SMOKE']

In [281]:
df_clean = df_cut.drop(smokecols, axis=1)

In [282]:
print("NANs for numeric variables")
[ (c, np.sum(np.isnan(df_clean[c])))  for c in df_clean.columns if np.issubdtype(df_clean[c].dtype, np.number) ]

NANs for numeric variables


[('COMPUTED WEIGHT IN KILOGRAMS', 21500),
 ('COMPUTED FRUIT JUICE INTAKE IN TIMES PER', 13362),
 ('COMPUTED FRUIT INTAKE IN TIMES PER DAY', 13717),
 ('COMPUTED DARK GREEN VEGETABLE INTAKE IN', 13386),
 ('COMPUTED POTATO SERVINGS PER DAY', 16794),
 ('COMPUTED OTHER VEGETABLE INTAKE IN TIMES', 17342),
 ('Cigarette_tax', 0)]

In [283]:
print("Records: ",len(df_clean))
df_clean = df_clean.dropna()
print("Records after cleaning Nan: ",len(df_clean))

Records:  429964
Records after cleaning Nan:  377435


In [284]:
print("Infintes for numeric variables")
[ (c, np.sum(np.isinf(df_clean[c])))  for c in df_clean.columns if np.issubdtype(df_clean[c].dtype, np.number) ]

Infintes for numeric variables


[('COMPUTED WEIGHT IN KILOGRAMS', 0),
 ('COMPUTED FRUIT JUICE INTAKE IN TIMES PER', 0),
 ('COMPUTED FRUIT INTAKE IN TIMES PER DAY', 0),
 ('COMPUTED DARK GREEN VEGETABLE INTAKE IN', 0),
 ('COMPUTED POTATO SERVINGS PER DAY', 0),
 ('COMPUTED OTHER VEGETABLE INTAKE IN TIMES', 0),
 ('Cigarette_tax', 0)]

In [285]:
df_clean.shape

(377435, 40)

In [286]:
df_clean.head()

,COMPUTED PHYSICAL HEALTH STATUS,COMPUTED MENTAL HEALTH STATUS,HIGH BLOOD PRESSURE CALCULATED VARIABLE,CHOLESTEROL CHECKED CALCULATED VARIABLE,HIGH CHOLESTEROL CALCULATED VARIABLE,LIFETIME ASTHMA CALCULATED VARIABLE,CURRENT ASTHMA CALCULATED VARIABLE,COMPUTED ASTHMA STATUS,COMPUTED PREFERRED RACE,CALCULATED NON-HISPANIC RACE INCLUDING M,...,COMPUTED POTATO SERVINGS PER DAY,COMPUTED OTHER VEGETABLE INTAKE IN TIMES,150 MINUTE PHYSICAL ACTIVITY CALCULATED,300 MINUTE PHYSICAL ACTIVITY CALCULATED,FLU SHOT CALCULATED VARIABLE,PNEUMONIA VACCINATION CALCULATED VARIABL,EVER BEEN TESTED FOR HIV CALCULATED VARI,STATE FIPS CODE,Cigarette_tax,smoke_type
0,Zero days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,Yes,No,No,Never,White,White only,...,14.0,100.0,150+ minutes (or vigorous equivalent minutes) ...,301+ minutes (or vigorous equivalent minutes)...,Yes,Yes,Yes,Alabama,0.675,N
1,Zero days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,No,No,No,Never,White,White only,...,0.0,100.0,150+ minutes (or vigorous equivalent minutes) ...,301+ minutes (or vigorous equivalent minutes)...,Yes,Yes,Yes,Alabama,0.675,N
2,Zero days when physical health not good,Zero days when mental health not good,No,Had cholesterol checked in past 5 years,Yes,No,No,Never,White,White only,...,0.0,14.0,0 minutes (or vigorous equivalent minutes) of ...,0 minutes (or vigorous equivalent minutes) of...,Yes,Yes,No,Alabama,0.675,N
4,14+ days when physical health not good,Zero days when mental health not good,No,Had cholesterol checked in past 5 years,No,Yes,Yes,Current,White,White only,...,14.0,10.0,0 minutes (or vigorous equivalent minutes) of ...,0 minutes (or vigorous equivalent minutes) of...,No,No,No,Alabama,0.675,N
5,1-13 days when physical health not good,Zero days when mental health not good,Yes,Had cholesterol checked in past 5 years,No,No,No,Never,White,White only,...,43.0,100.0,1-149 minutes (or vigorous equivalent minutes...,1-300 minutes (or vigorous equivalent minutes...,Yes,No,No,Alabama,0.675,S


### Explor the value_counts and convert to categorical data

In [287]:
for c in df_clean.columns:
    print(c)
    print(df_clean[c].value_counts())
    print()

COMPUTED PHYSICAL HEALTH STATUS
Zero days when physical health not good    231350
1-13 days when physical health not good     89039
14+ days when physical health not good      51159
Don�t know/Refused/Missing                   5887
Name: COMPUTED PHYSICAL HEALTH STATUS, dtype: int64

COMPUTED MENTAL HEALTH STATUS
Zero days when mental health not good    249912
1-13 days when mental health not good     81102
14+ days when mental health not good      41924
Don�t know/Refused/Missing                 4497
Name: COMPUTED MENTAL HEALTH STATUS, dtype: int64

HIGH BLOOD PRESSURE CALCULATED VARIABLE
No                                     224995
Yes                                    151664
Don�t know/Not Sure/Refused/Missing       776
Name: HIGH BLOOD PRESSURE CALCULATED VARIABLE, dtype: int64

CHOLESTEROL CHECKED CALCULATED VARIABLE
Had cholesterol checked in past 5 years             325408
Have never had cholesterol checked                   21913
Don�t know/Not Sure Or Refused/Missing       

Number of drinks per week              197271
Did not drink                          174881
Don�t know/Not sure/Refused/Missing      5283
Name: COMPUTED NUMBER OF DRINKS OF ALCOHOL BEV, dtype: int64

HEAVY ALCOHOL CONSUMPTION  CALCULATED VA
No                            349727
Yes                            22336
Don�t know/Refused/Missing      5372
Name: HEAVY ALCOHOL CONSUMPTION  CALCULATED VA, dtype: int64

COMPUTED FRUIT JUICE INTAKE IN TIMES PER
0.0       155764
100.0      56459
14.0       24688
29.0       22399
7.0        17438
3.0        17027
43.0       16056
2.0         8937
10.0        7743
17.0        6797
200.0       6522
57.0        6487
13.0        5094
33.0        4535
71.0        4190
50.0        3906
67.0        2161
300.0       2101
20.0        1771
23.0        1302
27.0         865
86.0         766
83.0         634
400.0        623
40.0         567
500.0        435
47.0         346
700.0        332
143.0        203
600.0         98
           ...  
110.0          3
2

2.000    44707
4.350    18758
1.339    18127
1.290    17570
3.040    14129
1.700    13825
0.570    13685
0.640    13374
0.995    11844
3.025    11160
0.600    11122
1.600    10384
1.410    10216
2.700     9608
0.300     8118
0.840     7838
2.870     7761
3.200     6953
1.360     6510
0.170     6430
0.440     6134
1.530     6070
2.600     5697
3.510     5654
0.675     5645
1.660     5554
3.080     5396
1.030     5276
1.980     5019
2.520     4978
0.370     4860
1.780     4727
1.200     4712
0.620     4671
4.250     4629
1.320     4409
1.150     4262
0.680     4247
0.450     4074
5.100     4026
1.080     3886
2.920     3530
2.100     3277
1.800     3252
3.000     1331
Name: Cigarette_tax, dtype: int64

smoke_type
N    316210
S     48998
B      6467
E      5760
Name: smoke_type, dtype: int64



In [288]:
print("Data types and unique values")
[ (df_clean[c].dtype.name,len(df_clean[c].value_counts()))  for c in df_clean.columns]

Data types and unique values


[('object', 4),
 ('object', 4),
 ('object', 3),
 ('object', 4),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('object', 4),
 ('object', 9),
 ('object', 9),
 ('object', 9),
 ('object', 3),
 ('object', 6),
 ('object', 6),
 ('object', 2),
 ('object', 2),
 ('float64', 547),
 ('object', 2),
 ('object', 5),
 ('object', 3),
 ('object', 7),
 ('object', 5),
 ('object', 6),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('object', 3),
 ('float64', 101),
 ('float64', 127),
 ('float64', 118),
 ('float64', 89),
 ('float64', 116),
 ('object', 4),
 ('object', 4),
 ('object', 4),
 ('object', 4),
 ('object', 4),
 ('object', 53),
 ('float64', 45),
 ('object', 4)]

In [289]:
df_categ = df_clean.copy()
for c in df_categ.columns:
    if df_categ[c].dtype.name == 'object':
        df_categ[c] = labelencoder.fit_transform(df_categ[c])
df_categ.head()

,COMPUTED PHYSICAL HEALTH STATUS,COMPUTED MENTAL HEALTH STATUS,HIGH BLOOD PRESSURE CALCULATED VARIABLE,CHOLESTEROL CHECKED CALCULATED VARIABLE,HIGH CHOLESTEROL CALCULATED VARIABLE,LIFETIME ASTHMA CALCULATED VARIABLE,CURRENT ASTHMA CALCULATED VARIABLE,COMPUTED ASTHMA STATUS,COMPUTED PREFERRED RACE,CALCULATED NON-HISPANIC RACE INCLUDING M,...,COMPUTED POTATO SERVINGS PER DAY,COMPUTED OTHER VEGETABLE INTAKE IN TIMES,150 MINUTE PHYSICAL ACTIVITY CALCULATED,300 MINUTE PHYSICAL ACTIVITY CALCULATED,FLU SHOT CALCULATED VARIABLE,PNEUMONIA VACCINATION CALCULATED VARIABL,EVER BEEN TESTED FOR HIV CALCULATED VARI,STATE FIPS CODE,Cigarette_tax,smoke_type
0,3,3,2,2,2,1,1,3,8,8,...,14.0,100.0,2,2,3,3,3,0,0.675,2
1,3,3,2,2,1,1,1,3,8,8,...,0.0,100.0,2,2,3,3,3,0,0.675,2
2,3,3,1,2,2,1,1,3,8,8,...,0.0,14.0,0,0,3,3,1,0,0.675,2
4,1,3,1,2,1,2,2,0,8,8,...,14.0,10.0,0,0,2,2,1,0,0.675,2
5,0,3,2,2,1,1,1,3,8,8,...,43.0,100.0,1,1,3,2,1,0,0.675,3


In [290]:
print(df_clean['smoke_type'].value_counts())

N    316210
S     48998
B      6467
E      5760
Name: smoke_type, dtype: int64


### Note: This data set has serious class imbalance issue

In [291]:
#Address class imbalance:
def classImballanceDownSample(df,ycol):
    df = df.copy()
    valueCount = df[ycol].value_counts()
    print("Before Class Imballance Treatment: ")
    print(valueCount)
    classes = valueCount.index
    counts = valueCount.values
    minClassSize = np.min(counts)
    for clas in classes:
        df1 = df[df[ycol]==clas]
        df2 = df[df[ycol]!=clas]
        
        df1 = df1.sample(n=minClassSize, random_state=50)
        df = df1.append(df2)
    #shuffling the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    print("After Class Imballance Treatment: ")
    print(df[ycol].value_counts())
    return df 

In [292]:
df_categ = classImballanceDownSample(df_categ, 'smoke_type')

Before Class Imballance Treatment: 
2    316210
3     48998
0      6467
1      5760
Name: smoke_type, dtype: int64
After Class Imballance Treatment: 
3    5760
2    5760
1    5760
0    5760
Name: smoke_type, dtype: int64


In [293]:
df_categ.shape

(23040, 40)

In [294]:
label = df_categ[['smoke_type']]
X = df_categ.drop(['smoke_type'], axis=1)

In [295]:
label.shape

(23040, 1)

In [296]:
X.shape

(23040, 39)

In [297]:
#0:Both; 1: E-smoker; 2: None 3: Smoker
for c in label.columns:
    print(c)
    print(label[c].value_counts())
    print()

smoke_type
3    5760
2    5760
1    5760
0    5760
Name: smoke_type, dtype: int64



# 2. Feature selection

### 2.1 Random Forest (RF) Classifier

In [298]:
X_train, X_test, Y_train, Y_test = train_test_split(X, label, test_size=0.33, random_state=100)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(15436, 39) (15436, 1) (7604, 39) (7604, 1)


In [299]:
#fit Random Forest and test the accuracy:
def RandomForest(x_train, y_train, x_test, y_test):   
    rf = RandomForestClassifier(n_estimators=100)
    rf.fit(x_train, y_train)   
    y_pred = rf.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    MSE_rf = mean_squared_error(y_test, y_pred)
    ACC_rf = (cm[0][0] + cm[1][1] + cm[2][2] + cm[3][3]) / (len(y_pred))
    #accuracy = metrics.accuracy_score(test_y, predictions)
    table_rf = [[' ','0', '1', '2', '3'],
             ['0', cm[0][0], cm[0][1], cm[0][2], cm[0][3]],
             ['1', cm[1][0], cm[1][1], cm[1][2], cm[1][3]],
             ['2', cm[2][0], cm[2][1], cm[2][2], cm[2][3]],
             ['3', cm[3][0], cm[3][1], cm[3][2], cm[3][3]]]
    #feature_importances = rf.feature_importances_
    #top5_rf = sorted(feature_importances, reverse=True)[:5]
    importances = pd.DataFrame({'feature':x_train.columns,'importance':np.round(rf.feature_importances_,5)})
    importances = importances.sort_values('importance',ascending=False).set_index('feature')
    
    print("The confusion matrix is:")
    print(tabulate(table_rf, tablefmt="fancy_grid", numalign = "center"))
    print('The Accuracy Rate is', ACC_rf)
    print('The Mean Squared Error is:', MSE_rf)
    print('-------------------------------------------------')
    print('The Classification Report is:')
    print(classification_report(y_test, y_pred))
    print('The Feature Importance is:')
    print(importances)
    #print(feature_importances)
    #print('The Top 5 important features are:')
    #print([(x_train.columns[feature_importances.tolist().index(top5_rf[i])], top5_rf[i]) for i in range(5)])

In [300]:
RandomForest(X_train, Y_train, X_test, Y_test)

/Users/A123/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


The confusion matrix is:
╒═══╤═════╤═════╤══════╤═════╕
│   │  0  │  1  │  2   │  3  │
├───┼─────┼─────┼──────┼─────┤
│ 0 │ 731 │ 524 │ 252  │ 391 │
├───┼─────┼─────┼──────┼─────┤
│ 1 │ 509 │ 711 │ 362  │ 299 │
├───┼─────┼─────┼──────┼─────┤
│ 2 │ 189 │ 394 │ 1134 │ 216 │
├───┼─────┼─────┼──────┼─────┤
│ 3 │ 538 │ 407 │ 466  │ 481 │
╘═══╧═════╧═════╧══════╧═════╛
The Accuracy Rate is 0.40202524986849025
The Mean Squared Error is: 2.0278800631246714
-------------------------------------------------
The Classification Report is:
             precision    recall  f1-score   support

          0       0.37      0.39      0.38      1898
          1       0.35      0.38      0.36      1881
          2       0.51      0.59      0.55      1933
          3       0.35      0.25      0.29      1892

avg / total       0.40      0.40      0.40      7604

The Feature Importance is:
                                          importance
feature                                             
COMPUTED WEIG

## 2.2 K-nearest-neighbour Classifier

In [318]:
def KNN(x, y, x_train, y_train, x_test, y_test):
    NN = int(np.sqrt(len(x_train)))
    how = 'distance'
    n_feats = len(x.columns)
    knn = neighbors.KNeighborsClassifier(NN, how)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    MSE_knn = mean_squared_error(y_test, y_pred)
    ACC_knn = (cm[0][0] + cm[1][1] + cm[2][2] + cm[3][3]) / (len(y_pred))
    table_knn = [[' ','0', '1', '2', '3'],
             ['0', cm[0][0], cm[0][1], cm[0][2], cm[0][3]],
             ['1', cm[1][0], cm[1][1], cm[1][2], cm[1][3]],
             ['2', cm[2][0], cm[2][1], cm[2][2], cm[2][3]],
             ['3', cm[3][0], cm[3][1], cm[3][2], cm[3][3]]]
    
    print("The confusion matrix is:")
    print(tabulate(table_knn, tablefmt="fancy_grid", numalign = "center"))
    print('The Accuracy Rate is', ACC_knn)
    print('The Mean Squared Error is:', MSE_knn)
    print('-------------------------------------------------')
    print('The Classification Report is:')
    print(classification_report(y_test, y_pred))
    #print('Feature  Accuracy')
    #for i in range(n_feats):
        #X = x[:, i].reshape(-1, 1)
        #scores = cross_val_score(knn, x, y)
        #print('%d        %g' % (x.columns[i], scores.mean()))

In [319]:
KNN(X, label, X_train, Y_train, X_test, Y_test)

/Users/A123/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


The confusion matrix is:
╒═══╤═════╤═════╤═════╤═════╕
│   │  0  │  1  │  2  │  3  │
├───┼─────┼─────┼─────┼─────┤
│ 0 │ 450 │ 510 │ 489 │ 449 │
├───┼─────┼─────┼─────┼─────┤
│ 1 │ 400 │ 532 │ 550 │ 399 │
├───┼─────┼─────┼─────┼─────┤
│ 2 │ 314 │ 483 │ 788 │ 348 │
├───┼─────┼─────┼─────┼─────┤
│ 3 │ 427 │ 498 │ 517 │ 450 │
╘═══╧═════╧═════╧═════╧═════╛
The Accuracy Rate is 0.291951604418727
The Mean Squared Error is: 2.3003682272488164
-------------------------------------------------
The Classification Report is:
             precision    recall  f1-score   support

          0       0.28      0.24      0.26      1898
          1       0.26      0.28      0.27      1881
          2       0.34      0.41      0.37      1933
          3       0.27      0.24      0.25      1892

avg / total       0.29      0.29      0.29      7604



## 2.3 Naive Bayes (NB) Classifier

In [369]:
def GNB(x_train, y_train, x_test, y_test):
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    y_pred = gnb.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    MSE_gnb = mean_squared_error(y_test, y_pred)
    ACC_gnb = (cm[0][0] + cm[1][1] + cm[2][2] + cm[3][3]) / (len(y_pred))
    table_gnb = [[' ','0', '1', '2', '3'],
             ['0', cm[0][0], cm[0][1], cm[0][2], cm[0][3]],
             ['1', cm[1][0], cm[1][1], cm[1][2], cm[1][3]],
             ['2', cm[2][0], cm[2][1], cm[2][2], cm[2][3]],
             ['3', cm[3][0], cm[3][1], cm[3][2], cm[3][3]]]
    
    print("The confusion matrix is:")
    print(tabulate(table_gnb, tablefmt="fancy_grid", numalign = "center"))
    print('The Accuracy Rate is', ACC_gnb)
    print('The Mean Squared Error is:', MSE_gnb)
    print('-------------------------------------------------')
    print('The Classification Report is:')
    print(classification_report(y_test, y_pred))

In [370]:
GNB(X_train, Y_train, X_test, Y_test)

The confusion matrix is:
╒═══╤═════╤═════╤══════╤═════╕
│   │  0  │  1  │  2   │  3  │
├───┼─────┼─────┼──────┼─────┤
│ 0 │ 634 │ 101 │ 541  │ 622 │
├───┼─────┼─────┼──────┼─────┤
│ 1 │ 531 │ 134 │ 649  │ 567 │
├───┼─────┼─────┼──────┼─────┤
│ 2 │ 245 │ 67  │ 1244 │ 377 │
├───┼─────┼─────┼──────┼─────┤
│ 3 │ 431 │ 87  │ 694  │ 680 │
╘═══╧═════╧═════╧══════╧═════╛
The Accuracy Rate is 0.3540241977906365
The Mean Squared Error is: 2.3219358232509206
-------------------------------------------------
The Classification Report is:
             precision    recall  f1-score   support

          0       0.34      0.33      0.34      1898
          1       0.34      0.07      0.12      1881
          2       0.40      0.64      0.49      1933
          3       0.30      0.36      0.33      1892

avg / total       0.35      0.35      0.32      7604



/Users/A123/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 2.4 Random guessing

In [326]:
def Guess(y):
    y_pred = np.random.permutation(y)
    cm = confusion_matrix(y, y_pred)
    MSE_guess = mean_squared_error(y, y_pred)
    ACC_guess = (cm[0][0] + cm[1][1] + cm[2][2] + cm[3][3]) / (len(y_pred))
    table_guess = [[' ','0', '1', '2', '3'],
             ['0', cm[0][0], cm[0][1], cm[0][2], cm[0][3]],
             ['1', cm[1][0], cm[1][1], cm[1][2], cm[1][3]],
             ['2', cm[2][0], cm[2][1], cm[2][2], cm[2][3]],
             ['3', cm[3][0], cm[3][1], cm[3][2], cm[3][3]]]
    
    print("The confusion matrix is:")
    print(tabulate(table_guess, tablefmt="fancy_grid", numalign = "center"))
    print('The Accuracy Rate is', ACC_guess)
    print('The Mean Squared Error is:', MSE_guess)
    print('-------------------------------------------------')
    print('The Classification Report is:')
    print(classification_report(y, y_pred))

In [327]:
Guess(label)

The confusion matrix is:
╒═══╤══════╤══════╤══════╤══════╕
│   │  0   │  1   │  2   │  3   │
├───┼──────┼──────┼──────┼──────┤
│ 0 │ 1471 │ 1423 │ 1421 │ 1445 │
├───┼──────┼──────┼──────┼──────┤
│ 1 │ 1455 │ 1419 │ 1432 │ 1454 │
├───┼──────┼──────┼──────┼──────┤
│ 2 │ 1442 │ 1465 │ 1453 │ 1400 │
├───┼──────┼──────┼──────┼──────┤
│ 3 │ 1392 │ 1453 │ 1454 │ 1461 │
╘═══╧══════╧══════╧══════╧══════╛
The Accuracy Rate is 0.25190972222222224
The Mean Squared Error is: 2.4844618055555556
-------------------------------------------------
The Classification Report is:
             precision    recall  f1-score   support

          0       0.26      0.26      0.26      5760
          1       0.25      0.25      0.25      5760
          2       0.25      0.25      0.25      5760
          3       0.25      0.25      0.25      5760

avg / total       0.25      0.25      0.25     23040



## 2.5 Compare the above results:

RF:
* The Accuracy Rate is 0.40202524986849025
* The Mean Squared Error is: 2.0278800631246714

KNN:
* The Accuracy Rate is 0.291951604418727
* The Mean Squared Error is: 2.3003682272488164

NB:
* The Accuracy Rate is 0.3540241977906365
* The Mean Squared Error is: 2.3219358232509206

Guess:
* The Accuracy Rate is 0.25190972222222224
* The Mean Squared Error is: 2.4844618055555556

Top 10 importance features based on RF:
* COMPUTED WEIGHT IN KILOGRAMS                 
* STATE FIPS CODE                             
* Cigarette_tax                                
* COMPUTED POTATO SERVINGS PER DAY            
* COMPUTED FRUIT INTAKE IN TIMES PER DAY       
* COMPUTED DARK GREEN VEGETABLE INTAKE IN      
* COMPUTED OTHER VEGETABLE INTAKE IN TIMES     
* COMPUTED FRUIT JUICE INTAKE IN TIMES PER     
* COMPUTED INCOME CATEGORIES                   
* COMPUTED LEVEL OF EDUCATION COMPLETED CA 

# 3. Build the logit model to do the interpretation

In [338]:
logit = df_categ[['COMPUTED WEIGHT IN KILOGRAMS',
                'STATE FIPS CODE',
                'Cigarette_tax',
                'COMPUTED POTATO SERVINGS PER DAY',
                'COMPUTED FRUIT INTAKE IN TIMES PER DAY',
                'COMPUTED DARK GREEN VEGETABLE INTAKE IN',
                'COMPUTED OTHER VEGETABLE INTAKE IN TIMES',
                'COMPUTED FRUIT JUICE INTAKE IN TIMES PER',
                'COMPUTED INCOME CATEGORIES',
                'COMPUTED LEVEL OF EDUCATION COMPLETED CA',
                'smoke_type']]

In [339]:
y_logit = logit[['smoke_type']]
x_logit = logit.drop(['smoke_type'], axis=1)

In [340]:
x_logit.shape, y_logit.shape

((23040, 10), (23040, 1))

In [363]:
from statsmodels.discrete.discrete_model import MNLogit

In [365]:
mnlogit = MNLogit(y_logit, x_logit).fit()
print(mnlogit.summary())

Optimization terminated successfully.
         Current function value: 1.368636
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:             smoke_type   No. Observations:                23040
Model:                        MNLogit   Df Residuals:                    23010
Method:                           MLE   Df Model:                           27
Date:                Sat, 08 Dec 2018   Pseudo R-squ.:                 0.01274
Time:                        14:05:48   Log-Likelihood:                -31533.
converged:                       True   LL-Null:                       -31940.
                                        LLR p-value:                5.098e-154
                            smoke_type=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
COMPUTED WEIGHT IN KILOGRAMS             -8.316e

## Interpret the coefficient...